In [18]:
# Import live code changes in
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os
import geopandas as gpd
import rasterio
import pandas as pd
from functools import reduce

from sovereign.utils import map_flopros_to_adm, union_gdfs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Set filepaths and provide data info

In [24]:
root = Path.cwd().parent # find project root
flopros_path = os.path.join(root, 'inputs', 'flood', 'protection', 'flopros-THA.shp')
adm_path = os.path.join(root, 'inputs', 'boundaries', 'admin', 'gadm36_THA_1.shp')
mapping_csv = os.path.join(root, 'inputs', 'flood', 'protection', 'flopros-adm1-map.csv')
basin_path = os.path.join(root, 'inputs', 'boundaries', 'basins', 'BA_THA_lev06.shp')#
output_path = os.path.join(root, 'outputs', 'boundaries', 'analysis_basins.gpkg')

In [25]:
# Load data
flopros = gpd.read_file(flopros_path)
adm = gpd.read_file(adm_path)
mapping_df = pd.read_csv(mapping_csv)
basins = gpd.read_file(basin_path)

In [26]:
# Map protection levels to the Admin Dataset
protection_levels = map_flopros_to_adm(mapping_df, flopros, adm)

In [27]:
# Merge protection levels with GADM dataset
# Rename the columns we want to merge
basins = basins.rename(columns={'HYBAS_ID': 'HYBAS_ID_06'})
protection_levels = protection_levels.rename(columns={'GID_1': 'flpr_gid_1'})
# Select relevant columns 
basins_selected = basins[['geometry', 'HYBAS_ID_06']]
protection_levels_selected = protection_levels[['geometry', 'flpr_gid_1', 'NAME', 'MerL_Riv']]
datasets = [basins_selected, protection_levels_selected]
merged = reduce(union_gdfs, datasets)

e:\projects\sovereign-risk\sovereign\utils.py:205: UserWarning: `keep_geom_type=True` in overlay resulted in 224 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  return gpd.overlay(gdf1, gdf2_copy, how='union')


In [28]:
# Clean dataset (remove all rows where there are no column values consistently accross the rows)
columns_to_check = ["HYBAS_ID_06", "flpr_gid_1"]
# Create a boolean mask for non-null and non-zero values
mask = (merged[columns_to_check].notnull() & (merged[columns_to_check] != 0)).all(axis=1)
# Filter the GeoDataFrame to keep only rows that meet the criteria
filtered_merged = merged.loc[mask]

In [29]:
# Save to file
filtered_merged.to_file(output_path)